<a href="https://colab.research.google.com/github/shubham1262/Reco_Engine/blob/main/Content_Type_Recommendation_System_(Matrix_Multiplication).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

#Create Dataset
-For illustration purpose, assuming 3 users and 6 movies with 4 features (genre)

In [ ]:
users=['Prashant','Shubham','Subhangi']
movies=['Pathan','Tiger','Top Gun','Dr Strange','Grown Ups','Grown Ups 2']
features=['Action','Comedy','Fiction','Sci-Fi']

User Movies Rating

In [ ]:
user_movies = tf.constant([
    [6,7,8,0,0,0],
    [0,0,0,0,10,10],
    [0,0,0,10,0,0],
], dtype=tf.float32)

movie_features = tf.constant([
    [1,1,0,0],
    [1,0,0,0],
    [1,1,0,0],
    [1,0,1,1],
    [0,1,0,1],
    [0,1,0,1]
], dtype=tf.float32)

# Getting User Feature Matrix
-Matrix multiplication to get User-> feature matrix from user-movie and movie-feature matrix

In [ ]:
user_feature=tf.matmul(user_movies,movie_features)
user_feature

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[21., 14.,  0.,  0.],
       [ 0., 20.,  0., 20.],
       [10.,  0., 10., 10.]], dtype=float32)>

Normalizing values across columns

In [ ]:
user_features=user_feature/tf.reduce_sum(user_feature,axis=0)
user_features

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0.67741936, 0.4117647 , 0.        , 0.        ],
       [0.        , 0.5882353 , 0.        , 0.6666667 ],
       [0.32258064, 0.        , 1.        , 0.33333334]], dtype=float32)>

In [ ]:
#Number of features is to give ranking a limit
user_feature_ranking=tf.nn.top_k(user_features,len(features))[1]
user_feature_ranking

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [3, 1, 0, 2],
       [2, 3, 0, 1]], dtype=int32)>

Ranking feature relevance for each user

In [ ]:
for i in range(len(users)):
  print("{}: {}".format(users[i],[features[int(index)] for index in user_feature_ranking[i]]))

Prashant: ['Action', 'Comedy', 'Fiction', 'Sci-Fi']
Shubham: ['Sci-Fi', 'Comedy', 'Action', 'Fiction']
Subhangi: ['Fiction', 'Sci-Fi', 'Action', 'Comedy']


#Similarity Score
-User-Movie Ratings (Dot product of user_feature and movie_feature matrix)

In [ ]:
user_ratings=tf.matmul(user_features,tf.transpose(movie_features))
user_ratings

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[1.089184  , 0.67741936, 1.089184  , 0.67741936, 0.4117647 ,
        0.4117647 ],
       [0.5882353 , 0.        , 0.5882353 , 0.6666667 , 1.254902  ,
        1.254902  ],
       [0.32258064, 0.32258064, 0.32258064, 1.655914  , 0.33333334,
        0.33333334]], dtype=float32)>

Ignore movies that were already rated

In [ ]:
users_unseen_movies = tf.equal(user_movies, tf.zeros_like(user_movies))
ignore_matrix = tf.zeros_like(user_movies)

In [ ]:
users_ratings_new = tf.where(
    users_unseen_movies,
    user_ratings, #Displays if condition passes from this matrix
    ignore_matrix) #Displays if condition fails from this matrix

In [ ]:
users_ratings_new

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.67741936, 0.4117647 ,
        0.4117647 ],
       [0.5882353 , 0.        , 0.5882353 , 0.6666667 , 0.        ,
        0.        ],
       [0.32258064, 0.32258064, 0.32258064, 0.        , 0.33333334,
        0.33333334]], dtype=float32)>

#Final Recommendation

In [ ]:
# 2 denotes how many recommendation/top n indexes you'd like to get
reco_index=tf.nn.top_k(users_ratings_new,2)[1]
reco_index

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 4],
       [3, 0],
       [4, 5]], dtype=int32)>

Display recommendations across users

In [ ]:
for i in range(len(users)):
  print("{}: {}".format(users[i],[movies[int(index)] for index in reco_index[i]]))

Prashant: ['Dr Strange', 'Grown Ups']
Shubham: ['Dr Strange', 'Pathan']
Subhangi: ['Grown Ups', 'Grown Ups 2']
